In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import sys
sys.path.append("../")
from utils.vol_surface import VolatilitySurface
from utils.option_math import calc_forward_price
from utils.data_cleaning import DataCleaner
import numpy as np
import gc
import random
import os 

SYMBOL="qqq"
MAXDATE = "2022-09-30"

df = pd.read_csv("../data/processed/qqq/2022_01_03.csv")

#df.QUOTE_DATE = pd.to_datetime(df.QUOTE_DATE)
#df = df[df.QUOTE_DATE>"2022-01-01"].reset_index(drop=True)
print(df.shape)
df.head()

(2868, 28)


,QUOTE_DATE,QUOTE_TIME_HOURS,UNDERLYING_LAST,EXPIRE_DATE,DTE,STRIKE,C_IV,C_LAST,C_SIZE,C_BID,...,P_LAST,P_SIZE,P_BID,P_ASK,P_VOLUME,P_DELTA,P_GAMMA,P_VEGA,P_THETA,P_RHO
0,2021-12-01,16.0,387.00,2022-01-03,33.0,245.0,NaN,0.0,12 x 12,141.42,...,0.00,383 x 50,0.12,0.17,NaN,-0.00633,0.00027,0.01957,-0.01724,-0.00186
1,2021-12-02,16.0,389.97,2022-01-03,32.0,245.0,1.04089,0.0,15 x 1,144.68,...,0.00,290 x 290,0.13,0.17,NaN,-0.00579,0.00021,0.01919,-0.01880,-0.00251
2,2021-12-03,16.0,383.27,2022-01-03,31.0,245.0,0.99010,0.0,17 x 17,137.91,...,0.25,137 x 666,0.18,0.23,20.0,-0.00815,0.00031,0.02490,-0.02418,-0.00267
3,2021-12-06,16.0,386.32,2022-01-03,28.0,245.0,1.04709,0.0,1 x 17,141.07,...,0.12,67 x 300,0.11,0.14,11.0,-0.00488,0.00021,0.01648,-0.01806,-0.00125
4,2021-12-07,16.0,397.87,2022-01-03,27.0,245.0,0.56943,0.0,8 x 8,152.51,...,0.12,10 x 680,0.05,0.06,0.0,-0.00279,0.00013,0.00837,-0.00895,-0.00079


## Load Underlying Data

In [2]:
underlying_price = pd.read_csv("../data/raw/qqq_underlying.csv")
underlying_price.columns = ["QUOTE_DATE","UNDERLYING_LAST"]
underlying_price.QUOTE_DATE = pd.to_datetime(underlying_price.QUOTE_DATE)

## The Data Cleaning Class

In [ ]:
cleaner = DataCleaner(underlying=underlying_price)
df_clean = cleaner.clean_data(df,expiration="2022-01-03")
df_clean.head()

## Cleaning Files

In [3]:
cleaner = DataCleaner(
    underlying=underlying_price,
    minDTE=0,
    maxDTE=40,
    minMoneyness=-0.7,
    maxMoneyness=0.7,
    calandar="NYSE",
    r=0 # So that the European option pricer works for American options.
)

In [4]:
expiry_files = os.listdir(f"../data/processed/{SYMBOL}/")
expiry_dates = pd.to_datetime([
    file.replace(".csv","").replace("_","-") for file in expiry_files
])
keep_file = expiry_dates<=MAXDATE
total = len(expiry_files)
clean_files = []
failed_files = []

for i in range(len(expiry_files)):
    if not keep_file[i]:
        os.remove(f"../data/processed/{SYMBOL}/{expiry_files[i]}")
    else:
        df = pd.read_csv(f"../data/processed/{SYMBOL}/{expiry_files[i]}")
        try:
            df = cleaner.clean_data(df,expiration=expiry_dates[i])
            clean_files.append(expiry_files[i])
            df.to_csv(f"../data/processed/{SYMBOL}/{expiry_files[i]}")
        except:
            failed_files.append(expiry_files[i])
        if i%10==0:
            print(f"Completed {i} out of {total}")
    gc.collect()        

Completed 0 out of 547
Completed 10 out of 547
Completed 20 out of 547


../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in true_divide
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/opions/european.py:46: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1) / (self._s * self._vol * sqrt(self._tte))
../utils/opions/european.py:102: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1)/(self._s*self._vol*sqrt(self._tte))
../utils/option_math.py:26: RuntimeWarning: invalid value encountered in true_divide
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))


Completed 30 out of 547


../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in true_divide
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/opions/european.py:46: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1) / (self._s * self._vol * sqrt(self._tte))
../utils/opions/european.py:102: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1)/(self._s*self._vol*sqrt(self._tte))
../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in true_divide
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/opions/european.py:46: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1) / (self._s * self.

Completed 40 out of 547
Completed 50 out of 547


...

Completed 510 out of 547


../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in true_divide
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/opions/european.py:46: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1) / (self._s * self._vol * sqrt(self._tte))
../utils/opions/european.py:102: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1)/(self._s*self._vol*sqrt(self._tte))
../utils/option_math.py:26: RuntimeWarning: invalid value encountered in true_divide
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))


Completed 520 out of 547


../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in true_divide
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/opions/european.py:46: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1) / (self._s * self._vol * sqrt(self._tte))
../utils/opions/european.py:102: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1)/(self._s*self._vol*sqrt(self._tte))


Completed 530 out of 547
Completed 540 out of 547


In [9]:
df = pd.read_csv(f"../data/processed/{SYMBOL}/2020_10_30.csv")
df = cleaner.clean_data(df,expiration="2020-10-30")
clean_files.append(expiry_files[i])
#            df.to_csv(f"../data/processed/{SYMBOL}/{expiry_files[i]}")

../utils/data_cleaning.py:198: RuntimeWarning: divide by zero encountered in log
  z=np.log(train.C_PRICE.to_numpy())
/home/balbert/Documents/devclub/BEV/venv/lib/python3.7/site-packages/scipy/interpolate/fitpack2.py:1176: UserWarning: 
The required storage space exceeds the available storage space: nxest
or nyest too small, or s too small.
The weighted least-squares spline corresponds to the current set of
knots.
  warnings.warn(message)
../utils/data_cleaning.py:205: RuntimeWarning: divide by zero encountered in log
  z=np.log(train.P_PRICE.to_numpy())
../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in true_divide
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/option_math.py:26: RuntimeWarning: divide by zero encountered in double_scalars
  d1 = (log(s/k)+(r+vol**2/2.)*tte)/(vol*sqrt(tte))
../utils/opions/european.py:46: RuntimeWarning: invalid value encountered in double_scalars
  gamma = norm.pdf(d1) / (self._s * self._vol * sqrt(self._tte))
../

In [7]:
print(failed_files)

['2020_10_30.csv', '2016_01_22.csv', '2018_04_27.csv', '2022_02_14.csv', '2022_06_15.csv', '2022_06_01.csv', '2018_01_05.csv', '2022_07_05.csv', '2019_02_08.csv', '2016_08_19.csv', '2020_07_02.csv', '2022_05_27.csv', '2019_06_21.csv', '2018_08_03.csv', '2018_11_02.csv', '2022_09_12.csv', '2018_11_23.csv', '2021_07_30.csv', '2020_08_14.csv', '2017_06_02.csv', '2021_10_06.csv', '2017_01_06.csv', '2022_06_24.csv', '2021_06_23.csv', '2020_03_20.csv', '2022_05_25.csv', '2016_09_16.csv', '2021_07_06.csv', '2021_06_28.csv', '2020_09_11.csv', '2020_01_10.csv', '2020_11_06.csv', '2022_02_25.csv', '2016_06_30.csv', '2017_10_27.csv', '2019_06_14.csv', '2019_01_03.csv', '2017_09_29.csv', '2022_04_04.csv', '2016_11_25.csv', '2020_10_09.csv', '2019_07_05.csv', '2021_05_24.csv', '2021_05_10.csv', '2022_07_22.csv', '2018_03_23.csv', '2022_03_04.csv', '2021_10_27.csv', '2020_05_08.csv', '2016_04_15.csv', '2020_12_18.csv', '2021_10_13.csv', '2022_08_26.csv', '2020_05_22.csv', '2022_06_22.csv', '2022_09_